In [19]:
import sys
sys.path.append('/users/kuba/code/aic/mil')

import mil_pytorch.mil as mil
from mil_pytorch.utils import eval_utils, data_utils, train_utils, create_bags_simple


import numpy
import torch
from torch.utils.data import DataLoader, SubsetRandomSampler
from sklearn.datasets import make_classification
import time

def create_model(input_len, n_neurons1, n_neurons2, n_neurons3):
    # Define neural networks for processing of data before and after aggregation
    prepNN1 = torch.nn.Sequential(
        torch.nn.Linear(input_len, n_neurons1, bias = True),
        torch.nn.ReLU(),
        # torch.nn.Linear(n_neurons1, n_neurons2, bias = True),
        # torch.nn.ReLU(),
    )

    afterNN1 = torch.nn.Sequential(
        torch.nn.Linear(n_neurons1, 1),
        torch.nn.Tanh()
    )

    # Define model ,loss function and optimizer
    model = torch.nn.Sequential(
        mil.BagModel_3d(prepNN1, afterNN1, torch.mean, device = device)
    ).double()

    # Move model to gpu if available
    model = model.to(device)

    return model

# --- CONFIG ---

# Configurations
n_neurons1 = 15
n_neurons2 = 15
n_neurons3 = 15
learning_rate = 1e-3
weight_decay = 1e-3
epochs = 4000
pos = 50
neg = 50
class_sep = 1.0
n_features = 10
max_instances = 15
batch_size = 5
patience = 4000
delta = 0

print('INFO: CONFIG -')
print('n_neurons1:\t{}\nn_neurons2:\t{}\nn_neurons3:\t{}\nlearning_rate:\t{}\nweight_decay:\t{}\nepochs:\t\t{}\npos:\t\t{}\nneg:\t\t{}\nclass_sep:\t{}\nn_features:\t{}\nmax_instances:\t{}\nbatch_size:\t{}\npatience:\t{}\ndelta:\t\t{}'.format(n_neurons1, n_neurons2, n_neurons3, learning_rate, weight_decay, epochs, pos, neg, class_sep, n_features, max_instances, batch_size, patience, delta))

# --- DATA ---

# Create data
source_data, source_labels = make_classification(n_samples = 20000, n_features = n_features, n_informative = n_features, n_redundant = 0, n_repeated = 0, n_classes = 10, class_sep = class_sep, n_clusters_per_class = 1)
data, ids, labels = create_bags_simple.create_bags(source_data, source_labels, pos = pos, neg = neg, max_instances = max_instances)
n_instances = data_utils.ids2n_instances(ids)
data_3d = data_utils.create_3d_data(instances = data, n_instances = n_instances)

print("INFO: data shape - ", data_3d.shape, len(labels))

# Check if gpu available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("INFO: Using device: {}".format(device))

# Move data to gpu (if available)
data_3d = data_3d.double().to(device)
n_instances = n_instances.to(device)
labels = labels.long().to(device)

# Convert labels from (1, 0) to (1, -1) for tanh
labels[labels == 0] = -1

# Create dataset and divide to train and test part
dataset = mil.MilDataset_3d(data_3d, n_instances, labels, normalize = True)

train_indices, valid_indices, test_indices = data_utils.data_split(dataset = dataset, valid_ratio = 0.2, test_ratio = 0.2, shuffle = True, stratify = True)

train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(valid_indices)
test_sampler = SubsetRandomSampler(test_indices)

# Create dataloaders
if batch_size == 0:
    train_dl = DataLoader(dataset, sampler = train_sampler, batch_size = len(train_indices))
else:
    train_dl = DataLoader(dataset, sampler = train_sampler, batch_size = batch_size)

valid_dl = DataLoader(dataset, sampler = valid_sampler, batch_size = len(valid_indices))
test_dl = DataLoader(dataset, sampler = test_sampler, batch_size = len(test_indices))

# --- MODEL ---

model = create_model(len(dataset.data[0][0]) , n_neurons1, n_neurons2, n_neurons3)
criterion = mil.MyHingeLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate, weight_decay = weight_decay)


# --- TRAIN ---

train_utils.train_model(model, criterion, optimizer, train_dl, valid_dl, epochs, patience, delta)

# --- EVAL ---

eval_utils.evaluation(model, criterion, train_dl, test_dl, device)


INFO: CONFIG -
n_neurons1:	15
n_neurons2:	15
n_neurons3:	15
learning_rate:	0.001
weight_decay:	0.001
epochs:		4000
pos:		50
neg:		50
class_sep:	1.0
n_features:	10
max_instances:	15
batch_size:	5
patience:	4000
delta:		0
INFO: data shape -  torch.Size([100, 15, 10]) 100
INFO: Using device: cpu
TRAINING:


/users/kuba/code/aic/mil/mil_pytorch/utils/data_utils.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data[i] = torch.cat([ torch.tensor(instances[ marker : marker + n_instances[i] ]) ,  torch.zeros(max_n_instances - n_instances[i], n_features, dtype = torch.float) ], dim = 0)


[100/4000] | train_loss: 0.6336758732795715 | valid_loss: 0.8242297172546387 
[200/4000] | train_loss: 0.4113631546497345 | valid_loss: 0.813481330871582 
[300/4000] | train_loss: 0.2602430582046509 | valid_loss: 0.8234900832176208 
[400/4000] | train_loss: 0.18170462548732758 | valid_loss: 0.8219959139823914 
[500/4000] | train_loss: 0.14510203897953033 | valid_loss: 0.7961704730987549 
[600/4000] | train_loss: 0.12764887511730194 | valid_loss: 0.7721650004386902 
[700/4000] | train_loss: 0.11857381463050842 | valid_loss: 0.7600780129432678 
[800/4000] | train_loss: 0.11418673396110535 | valid_loss: 0.7551567554473877 
[900/4000] | train_loss: 0.11198247224092484 | valid_loss: 0.7547878623008728 
[1000/4000] | train_loss: 0.11090278625488281 | valid_loss: 0.7555310726165771 
[1100/4000] | train_loss: 0.11044617742300034 | valid_loss: 0.7558886408805847 
[1200/4000] | train_loss: 0.11000183969736099 | valid_loss: 0.7497093677520752 
[1300/4000] | train_loss: 0.10985008627176285 | valid

In [16]:
data_3d.shape

torch.Size([100, 15, 10])

In [17]:
n_instances.shape

torch.Size([100])

In [18]:
for data, n_instances, labels in train_dl:
    print(data.shape)
    print(n_instances.shape)
    print(labels.shape)
    print('\n')

torch.Size([5, 15, 10])
torch.Size([5])
torch.Size([5])


torch.Size([5, 15, 10])
torch.Size([5])
torch.Size([5])


torch.Size([5, 15, 10])
torch.Size([5])
torch.Size([5])


torch.Size([5, 15, 10])
torch.Size([5])
torch.Size([5])


torch.Size([5, 15, 10])
torch.Size([5])
torch.Size([5])


torch.Size([5, 15, 10])
torch.Size([5])
torch.Size([5])


torch.Size([5, 15, 10])
torch.Size([5])
torch.Size([5])


torch.Size([5, 15, 10])
torch.Size([5])
torch.Size([5])


torch.Size([5, 15, 10])
torch.Size([5])
torch.Size([5])


torch.Size([5, 15, 10])
torch.Size([5])
torch.Size([5])


torch.Size([5, 15, 10])
torch.Size([5])
torch.Size([5])


torch.Size([5, 15, 10])
torch.Size([5])
torch.Size([5])




In [10]:
len(dataset.data[0][0])

10